In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [4]:
df=pd.read_csv("Mumbai-House- Prices.csv")
#df.info()
# step-1 Data Preprocessing ,Remove Useless columns ,change some values
# step-2 After Data Preprocess apply Linear Model and check some other model also
# step-3 Create a Pipeline
# step-4 Import Pipline using pkl

In [5]:
    # Here we are remove useless columns which is not used in prediction
df=pd.read_csv("Mumbai-House- Prices.csv")
df.drop(columns = ["status" ,"age"] ,inplace=True)
df = df[["region" ,"locality" ,"type" ,"bhk" ,"area" ,"price" ,"price_unit"]]
#Handdle Missing and Duplicate values
df.isnull().sum()
df.duplicated().sum()
df["region"].value_counts()
df["locality"].value_counts()
(df["locality"].value_counts() > 30).sum()
df["price_unit"].value_counts()
(df["region"].value_counts() <= 10).sum()
categories = df["region"].value_counts() <= 10
changeable_categories = categories[categories == True]
changeable_categories

region
Koproli                True
Sector 20 Kamothe      True
Kanjurmarg East        True
Vasind                 True
Dahisar West           True
                       ... 
Adaigaon               True
Sector-4 New Panvel    True
Versova                True
Sector 6 Kamothe       True
Pen                    True
Name: count, Length: 114, dtype: bool

In [6]:
df["region"] = df["region"].apply(lambda x : "others" if x in changeable_categories else x)

In [7]:
df["region"].value_counts()

region
Thane West         14868
Mira Road East      9902
Dombivali           3041
Kandivali East      2568
Kharghar            2362
                   ...  
Mazgaon               16
Nalasopara East       15
Kalwa                 15
Vangani               13
Saphale               12
Name: count, Length: 115, dtype: int64

In [8]:
df.drop(columns=["locality"] ,inplace=True)
df.head()

,region,type,bhk,area,price,price_unit
0,Andheri West,Apartment,3,685,2.50,Cr
1,Naigaon East,Apartment,2,640,52.51,L
2,Borivali West,Apartment,2,610,1.73,Cr
3,Panvel,Apartment,2,876,59.98,L
4,Mira Road East,Apartment,2,659,94.11,L


In [9]:
df["price"] = df["price"].astype(str) + " " + df["price_unit"]
df.head()

,region,type,bhk,area,price,price_unit
0,Andheri West,Apartment,3,685,2.5 Cr,Cr
1,Naigaon East,Apartment,2,640,52.51 L,L
2,Borivali West,Apartment,2,610,1.73 Cr,Cr
3,Panvel,Apartment,2,876,59.98 L,L
4,Mira Road East,Apartment,2,659,94.11 L,L


In [10]:
df["area"].skew()
df["bhk"].skew()
Q1 = df["area"].quantile(0.25)
Q3 = df["area"].quantile(0.75)

IQR = Q3 - Q1

Min = Q1 - 1.5 * IQR
Max = Q3 + 1.5 * IQR
print(Max)

df = df[(df["area"] < Max) & (df["area"] > Min)]

1987.5


In [11]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression ,Lasso ,ridge_regression
from sklearn.metrics import r2_score,accuracy_score

from sklearn.preprocessing import OneHotEncoder ,StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline,make_pipeline

In [12]:
X = df.iloc[: ,:4]
Y = df.iloc[: ,-1]

x_train,x_test,y_train,y_test = train_test_split(X ,Y ,test_size=0.2 ,random_state=2)
x_train.shape


(56847, 4)

In [13]:
x_test.shape

(14212, 4)

In [14]:
column_transform = ColumnTransformer([
    ("transform" ,OneHotEncoder(sparse_output=False ,drop="first") ,[0,1])
],remainder="passthrough")

In [15]:
scaler = StandardScaler()

In [16]:
lr = LinearRegression()

In [19]:
pipe = make_pipeline(column_transform ,scaler ,lr)

In [20]:
pipe.fit(x_train,y_train)

ValueError: could not convert string to float: 'Cr'

In [ ]:
y_pred = pipe.predict(x_test)